In [1]:
select version();

version()
10.7.0-MariaDB-MDEV-25015-sformat


# Create test database

In [2]:
create or replace database preview_sformat;

Query OK

In [3]:
use preview_sformat;

Query OK

# Create a small dataset

In [4]:
create or replace table employees (first_name varchar(100), last_name varchar(100), salary int, currency char(3));

Query OK

In [5]:
insert into employees values
('John', 'Doe', 100000, 'USD'),
('Harry', 'Smith', 98765, 'GBP'),
('Tom', 'Tom', 1000000, 'ZWD');

Query OK

# Pre-10.7 string formatting

### Regular concatenation is cool, but it can quickly get out of hand

In [6]:
select concat(first_name, ' ', last_name) from employees;

"concat(first_name, ' ', last_name)"
John Doe
Harry Smith
Tom Tom


In [7]:
select concat(first_name, ' ', last_name, ' -> ', cast(format(salary, 0) as varchar(10)), ' ', currency) from employees;

"concat(first_name, ' ', last_name, ' -> ', cast(format(salary, 0) as varchar(10)), ' ', currency)"
"John Doe -> 100,000 USD"
"Harry Smith -> 98,765 GBP"
"Tom Tom -> 1,000,000 ZWD"


# 10.7 SFORMAT-preview string formatting

In [8]:
select sformat('{} {} -> {:L} {}', first_name, last_name, salary, currency) from employees;

"sformat('{} {} -> {:L} {}', first_name, last_name, salary, currency)"
"John Doe -> 100,000 USD"
"Harry Smith -> 98,765 GBP"
"Tom Tom -> 1,000,000 ZWD"


### Creating a reports dataset

In [9]:
create or replace table reports (table_name varchar(100), cond_column_name varchar(100), value varchar(100));

Query OK

In [10]:
insert into reports values ('employees', 'currency', 'usd');

Query OK

### Dynamic SQL generation becomes trivial

In [11]:
select sformat('select * from `{}` where `{}` = {}', table_name, cond_column_name, quote(value)) from reports;

"sformat('select * from `{}` where `{}` = {}', table_name, cond_column_name, quote(value))"
select * from `employees` where `currency` = 'usd'


In [12]:
select sformat('select * from `{}` where `{}` = {}', table_name, cond_column_name, quote(value)) into @q from reports;

Query OK

In [13]:
execute immediate @q;

first_name,last_name,salary,currency
John,Doe,100000,USD


# More SFORMAT examples
### Enjoy playing around with the new function and experiment with various corner cases

In [14]:
create table t1(rel char(6), mdev int, feature char(20));

Query OK

In [15]:
insert into t1 values('10.7.0', 25015, 'Python style sformat'), 
                     ('10.7.0', 4958, 'UUID');

Query OK

In [16]:
select * from t1;

rel,mdev,feature
10.7.0,25015,Python style sformat
10.7.0,4958,UUID


# Show simple sformat usage

In [17]:
select sformat('MariaDB Server {} has a preview for MDEV-{} which is about {}', 
       rel, mdev, feature) as 'Preview Release Examples' from t1;

Preview Release Examples
MariaDB Server 10.7.0 has a preview for MDEV-25015 which is about Python style sformat
MariaDB Server 10.7.0 has a preview for MDEV-4958 which is about UUID


# Test case with some intentional bugs

In [18]:
create or replace table t1 (param1 text, param2 text, param3 int);

Query OK

In [19]:
insert into  t1 values ('string: {}, number: {}', 'xyz', 123),
                       ('something {} went {} wrong {}', 'foo', 0),
                       ('Test case {} -> {}', 'Print', -32);

Query OK

In [20]:
select sformat(param1, param2, param3) from t1;

"sformat(param1, param2, param3)"
"string: xyz, number: 123"
NULL
Test case Print -> -32


# FLOATs and DATEs

In [21]:
create or replace table t2 (param1 FLOAT, param2 SMALLINT, param3 CHAR, param4 DATE);

Query OK

In [22]:
insert into t2 values (0.0025, 25, 'A', DATE('2020-06-29')),
                      (0.0005,  5, 'B', DATE('2020-6-29')),
                      (5.5555, -5, 'C', DATE('200629')),
                      (-9,     -9, 'D', DATE('20*06*29'));

Query OK

In [23]:
select sformat('p1 {:.4f} p2 {} p3 {} p4 {}', param1, param2, param3, param4) from t2;

"sformat('p1 {:.4f} p2 {} p3 {} p4 {}', param1, param2, param3, param4)"
p1 0.0025 p2 25 p3 A p4 2020-06-29
p1 0.0005 p2 5 p3 B p4 2020-06-29
p1 5.5555 p2 -5 p3 C p4 2020-06-29
p1 -9.0000 p2 -9 p3 D p4 2020-06-29


# utf8 vs. latin1

In [24]:
set names utf8;

Query OK

In [25]:
create or replace table t3 (format_str   text character set 'latin1',
                            first_param  text character set 'utf8',
                            second_param text character set 'latin1');

Query OK

In [26]:
insert into t3 values ('test 1 {} {}', UNHEX('C3A5'), UNHEX('E5'));

Query OK

In [27]:
select sformat(format_str, first_param, second_param) from t3;

"sformat(format_str, first_param, second_param)"
test 1 å å


In [28]:
select HEX(sformat(format_str, first_param, second_param)) from t3;

"HEX(sformat(format_str, first_param, second_param))"
74657374203120C3A520C3A5


# bit, boolean, decimal, timestamp

In [29]:
set names latin1;

Query OK

In [30]:
create or replace table t4 (p1 bit(8), p2 boolean, p3 DECIMAL, p4 TIMESTAMP);

Query OK

In [31]:
insert into t4 values (42, TRUE,  42, '2021-08-18 16:50:07'),
                      (24, FALSE, 24, '0000-00-00 00:00:00');

Query OK

In [32]:
select sformat('{}: {} {} {} {}', 'Data', p1, p2, p3, p4) from t4;

"sformat('{}: {} {} {} {}', 'Data', p1, p2, p3, p4)"
Data: 42 1 42 2021-08-18 16:50:07
Data: 24 0 24 0000-00-00 00:00:00


# HEX and more utf8

In [33]:
set names utf8;

Query OK

In [34]:
create or replace table t5 (param text character SET utf8 COLLATE utf8_general_ci);

Query OK

In [35]:
insert into t5 values (UNHEX('C3A5')),(UNHEX('C5BB')),(UNHEX('e0b1bb')),
                      (UNHEX('C38A')), (NULL);

Query OK

In [36]:
select sformat('{}', param) from t5;

"sformat('{}', param)"
å
Ż
౻
Ê
NULL


In [37]:
select HEX(sformat('{}', param)) from t5;

"HEX(sformat('{}', param))"
C3A5
C5BB
E0B1BB
C38A
NULL


# A large number of sformat examples

In [38]:
select sformat('Num {:L}', 13800000000);

"sformat('Num {:L}', 13800000000)"
"Num 13,800,000,000"


In [39]:
select sformat('Num [{:20}]', 42);

"sformat('Num [{:20}]', 42)"
Num [ 42]


In [40]:
select sformat('Number: {:*^{}}', 4, 5);

"sformat('Number: {:*^{}}', 4, 5)"
Number: **4**


In [41]:
select sformat('{:02} - {:02} - {:02}', 1, 2, 3);

"sformat('{:02} - {:02} - {:02}', 1, 2, 3)"
01 - 02 - 03


In [42]:
select sformat('Character {:c}', 'h');

"sformat('Character {:c}', 'h')"
Character h


In [43]:
select sformat('String {:s}', 'hello');

"sformat('String {:s}', 'hello')"
String hello


In [44]:
select sformat('Large {0:+010.4g}', 392.64);

"sformat('Large {0:+010.4g}', 392.64)"
Large +0000392.6


In [45]:
select sformat('Large {:g}', 392.65);

"sformat('Large {:g}', 392.65)"
Large 392.65


In [46]:
select sformat('Float {:.2f}', 42.0);

"sformat('Float {:.2f}', 42.0)"
Float 42.00


In [47]:
select sformat('Float {:f}', 42.0);

"sformat('Float {:f}', 42.0)"
Float 42.000000


In [48]:
select sformat('Number {:d}', 42);

"sformat('Number {:d}', 42)"
Number 42


In [49]:
select sformat('Number {:{}}', 5, 5);

"sformat('Number {:{}}', 5, 5)"
Number 5


In [50]:
select sformat('Number [{:10}]', 9999);

"sformat('Number [{:10}]', 9999)"
Number [ 9999]


In [51]:
select sformat('Number {:.3}', 3.1416);

"sformat('Number {:.3}', 3.1416)"
Number 3.14


In [52]:
select sformat('int: {0:d};  hex: {0:x};  oct: {0:o}', 42);

"sformat('int: {0:d}; hex: {0:x}; oct: {0:o}', 42)"
int: 42; hex: 2a; oct: 52


In [53]:
select sformat('int: {0:d};  hex: {0:#x};  oct: {0:#o}', 42);

"sformat('int: {0:d}; hex: {0:#x}; oct: {0:#o}', 42)"
int: 42; hex: 0x2a; oct: 052


In [54]:
select sformat('The hexadecimal version of {0} is {0:X}', 255);

"sformat('The hexadecimal version of {0} is {0:X}', 255)"
The hexadecimal version of 255 is FF


In [55]:
select sformat('The hexadecimal version of {0} is {0:x}', 255);

"sformat('The hexadecimal version of {0} is {0:x}', 255)"
The hexadecimal version of 255 is ff


In [56]:
select sformat('The octal version of {0} is {0:o}', 10);

"sformat('The octal version of {0} is {0:o}', 10)"
The octal version of 10 is 12


In [57]:
select sformat('The binary version of {0} is {0:b}', 5);

"sformat('The binary version of {0} is {0:b}', 5)"
The binary version of 5 is 101


In [58]:
select sformat('{:+f}; {:+f}', 3.14, -3.14);

"sformat('{:+f}; {:+f}', 3.14, -3.14)"
+3.140000; -3.140000


In [59]:
select sformat('{: f}; {: f}', 3.14, -3.14);

"sformat('{: f}; {: f}', 3.14, -3.14)"
3.140000; -3.140000


In [60]:
select sformat('{:-f}; {:-f}', 3.14, -3.14);

"sformat('{:-f}; {:-f}', 3.14, -3.14)"
3.140000; -3.140000


In [61]:
select sformat('The temperature is between {: } and {: } degrees celsius.', -3, 7);

"sformat('The temperature is between {: } and {: } degrees celsius.', -3, 7)"
The temperature is between -3 and 7 degrees celsius.


In [62]:
select sformat('The temperature is between {:-} and {:-} degrees celsius.', -3, 7);

"sformat('The temperature is between {:-} and {:-} degrees celsius.', -3, 7)"
The temperature is between -3 and 7 degrees celsius.


In [63]:
select sformat('The temperature is between {:+} and {:+} degrees celsius.', -3, 7);

"sformat('The temperature is between {:+} and {:+} degrees celsius.', -3, 7)"
The temperature is between -3 and +7 degrees celsius.


In [64]:
select sformat('We have {:<8} chickens.', 49);

"sformat('We have {:<8} chickens.', 49)"
We have 49 chickens.


In [65]:
select sformat('Center align [{:*^10}]', 'data');

"sformat('Center align [{:*^10}]', 'data')"
Center align [***data***]


In [66]:
select sformat('Center align [{:^10}].', 'data');

"sformat('Center align [{:^10}].', 'data')"
Center align [ data ].


In [67]:
select sformat('Right align [{:>10}].', 'data');

"sformat('Right align [{:>10}].', 'data')"
Right align [ data].


In [68]:
select sformat('Left align [{:<10}].', 'data');

"sformat('Left align [{:<10}].', 'data')"
Left align [data ].


In [69]:
select sformat('{0}{1}{0}', 'abra', 'cad');

"sformat('{0}{1}{0}', 'abra', 'cad')"
abracadabra


In [70]:
select sformat('Change Order {1} {0}', 'second', 'first');

"sformat('Change Order {1} {0}', 'second', 'first')"
Change Order first second
